# Setup


**Imports**


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from loadmydata.load_human_locomotion import (
    load_human_locomotion_dataset,
    get_code_list,
)

rng = np.random.default_rng()


**Utility functions**


In [ ]:
phi = (1 + 5**0.5) / 2  # golden ratio ≈ 1.618


def fig_ax(figsize=(phi * 5, 5)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.autoscale(enable=True, axis="x", tight=True)
    return fig, ax

# Spectral feature


## Question 6


In [ ]:
from statsmodels.tsa.stattools import acovf
from scipy.signal import periodogram


def sim_plot_acov_periodogram():
    # simulation parameters
    Ns = [200, 500, 1000]
    N_sim = 100
    fs = 1.0

    for N in Ns:
        # define max lag
        max_lag = N - 1
        acovs = np.zeros((N_sim, max_lag + 1))
        for s in range(N_sim):
            x = rng.normal(0, 1, N)
            # compute biased estimate because less variance
            ac = acovf(x, demean=False, fft=True, adjusted=False)
            acovs[s] = ac[: max_lag + 1]

        mu, std = acovs.mean(0), acovs.std(0)
        lags = np.arange(max_lag + 1)

        fig, ax = fig_ax()
        ax.plot(lags, mu, lw=2, label="mean")
        ax.fill_between(lags, mu - std, mu + std, alpha=0.25, label="±1 std")
        ax.set_title(f"ACov white noise (N={N}, fs={fs} Hz, {N_sim} runs)")

        plt.xlabel("Lag τ")
        plt.ylabel(r"$\hat{\gamma}(\tau)$")
        plt.grid(alpha=0.4)
        plt.legend()
        plt.tight_layout()
        # save figure in figures
        plt.savefig(f"figures/acov_white_noise_N{N}_fs{fs}_sim{N_sim}.png")
        plt.show()

    for N in Ns:
        f, _ = periodogram(np.zeros(N), fs=fs)
        Pxx = np.zeros((N_sim, len(f)))

        for s in range(N_sim):
            x = rng.normal(0, 1, N)
            f, Pxx[s] = periodogram(x, fs=fs, scaling="density", return_onesided=True)
            # rescale Pxx because periodogram returns one-sided PSD
            Pxx[s] /= 2

        mu, std = Pxx.mean(0), Pxx.std(0)

        fig, ax = fig_ax()
        ax.plot(f, mu, lw=2, label="mean")
        ax.fill_between(f, mu - std, mu + std, alpha=0.25, label="±1 std")
        ax.set_title(f"Periodogram of white noise (N={N}, fs={fs} Hz, {N_sim} runs)")
        plt.xlabel("Frequency f (Hz)")
        plt.ylabel("Power spectral density")
        plt.grid(alpha=0.4)
        plt.legend()
        plt.tight_layout()
        # save figure in figures
        plt.savefig(f"figures/periodogram_white_noise_N{N}_fs{fs}_sim{N_sim}.png")
        plt.show()


sim_plot_acov_periodogram()

## Question 9


In [ ]:
def sim_plot_periodogram_bartlett():
    Ns = [200, 500, 1000]
    N_sim = 100
    fs = 1.0
    K = 5

    for N in Ns:
        N_seg = N // K
        f, _ = periodogram(np.zeros(N_seg), fs=fs)  # correct frequency grid
        Pxx = np.zeros((N_sim, len(f)))

        for s in range(N_sim):
            x = rng.normal(0, 1, N)
            x_segments = np.array_split(x, K)
            Pxx_segments = []
            for segment in x_segments:
                f, Pxx_seg = periodogram(
                    segment, fs=fs, scaling="density", return_onesided=True
                )
                Pxx_seg /= 2  # rescale for one-sided PSD
                Pxx_segments.append(Pxx_seg)
            Pxx[s] = np.mean(Pxx_segments, axis=0)

        mu, std = Pxx.mean(0), Pxx.std(0)

        fig, ax = fig_ax()
        ax.plot(f, mu, lw=2, label="mean")
        ax.fill_between(f, mu - std, mu + std, alpha=0.25, label="±1 std")
        ax.set_title(f"Bartlett Periodogram (N={N}, K={K}, {N_sim} sims)")
        ax.set_xlabel("Frequency f (Hz)")
        ax.set_ylabel("Power spectral density")
        ax.grid(alpha=0.4)
        ax.legend()
        plt.tight_layout()
        # save figure in figures
        plt.savefig(
            f"figures/bartlett_periodogram_white_noise_N{N}_K{K}_fs{fs}_sim{N_sim}.png"
        )
        plt.show()


sim_plot_periodogram_bartlett()

# Dynamic time warping (DTW)


## Data

This data set consists of signals collected with inertial measurement units (accelerometer+gyroscope), from 230 subjects undergoing a fixed protocol:

- standing still,
- walking 10 m,
- turning around,
- walking back,
- stopping.

In this assignment, we only consider the vertical acceleration of the left foot and all signals are truncated to 20 seconds (as a result, they all have same length). Signals are sampled at 100 Hz.

The measured population is composed of healthy subjects as well as patients with neurological or orthopedic disorders.

The start and end time stamps of thousands of footsteps are available.

The data are part of a larger data set described in [1].

[1] Truong, C., Barrois-Müller, R., Moreau, T., Provost, C., Vienne-Jumeau, A., Moreau, A., Vidal, P.-P., Vayatis, N., Buffat, S., Yelnik, A., Ricard, D., & Oudre, L. (2019). A data set for the study of human locomotion with inertial measurements units. Image Processing On Line (IPOL), 9.


**The task** is to classify footsteps in healthy/non-healthy.


The following cell defines the training set `(X_train, y_train)` and testing set `(X_test, y_test)`.


In [ ]:
subset_indexes_train = [95, 619, 441, 149, 951, 803, 214, 34, 37, 630]
subset_indexes_test = [683, 259, 59, 387, 634]

code_list = get_code_list()

X_train = list()  # list of footstep signals
y_train = list()  # list of pathologies (the "labels")

for code in np.take(code_list, subset_indexes_train):
    single_trial = load_human_locomotion_dataset(code)
    signal = (
        single_trial.signal.LAZ.to_numpy()
    )  # keeping only one dimension (from the left sensor)
    steps = single_trial.left_steps
    pathology = single_trial.metadata["PathologyGroup"]
    label = 0 if pathology == "Healthy" else 1  # 0: healthy, 1: non-healthy
    for start, end in steps:
        X_train.append(signal[start:end])
        y_train.append(label)


X_test = list()  # list of footstep signals
y_test = list()  # list of pathologies (the "labels")

for code in np.take(code_list, subset_indexes_test):
    single_trial = load_human_locomotion_dataset(code)
    signal = (
        single_trial.signal.LAZ.to_numpy()
    )  # keeping only one dimension (from the left sensor)
    steps = single_trial.left_steps
    pathology = single_trial.metadata["PathologyGroup"]
    label = 0 if pathology == "Healthy" else 1  # 0: healthy, 1: non-healthy
    for start, end in steps:
        X_test.append(signal[start:end])
        y_test.append(label)

In [ ]:
len(X_train), len(X_test)

## Question 10


## Question 11
